# Production System Assignment

In [24]:
from __future__ import annotations

import random
import statistics
from collections.abc import Callable, Sequence

import numpy as np
import simpy
from scipy import stats
from simpy.events import ProcessGenerator
from lib.server import Server
from lib.job import Job

In [29]:
class System:
    def __init__(
        self,
        env: simpy.Environment,
        inter_arrival_time_distribution: Callable[[], float],
        processing_time_per_family_distribution: list[Callable[[], float]],
        families_distribution: Callable[[], float],
        due_dates_distribution: Callable[[], float],
        routing_distribution: dict[int, list[Callable[[], float]]],
        routing_prob: dict[int, list[float]]
    ) -> None:
        self.env = env
        self.inter_arrival_time_distribution = inter_arrival_time_distribution
        self.processing_time_per_family_distribution = processing_time_per_family_distribution
        self.families_distribution = families_distribution
        self.due_dates_distribution = due_dates_distribution
        self.routing_distribution = routing_distribution
        self.routing_prob = routing_prob

        self.machines: list[Server] = []
        for i in range(6):
            server = Server(self.env, 1, "WC{}".format(i+1))
            self.machines.append(server)

        self.jobs: list[Job] = []
        self.env.process(self.run())

    @property
    def finished_jobs(self) -> int:
        return sum(job.done for job in self.jobs)

    def run(self) -> ProcessGenerator:
        idx = 0
        while True:
            timeout_inter_arrival = self.inter_arrival_time_distribution()
            weight = self.families_distribution()
            if weight <= 0.1:
                family = 1
            elif weight <= 0.62:
                family = 2
            else:
                family = 3
            processing_time = self.processing_time_per_family_distribution[family-1]()

            family_routing_distr = self.routing_distribution[family]
            family_routing_prob = self.routing_prob[family]

            job_routing = []
            for i in range(6):
                if family_routing_distr[i]() <= family_routing_prob[i]:
                    job_routing.append(self.machines[i])

            yield self.env.timeout(timeout_inter_arrival)

            job = Job(
                env=self.env,
                routing=job_routing,
                arrival_time=self.env.now,
                process_time=processing_time,
                idx=idx,
                family="F{}".format(family)
            )

            idx += 1
            self.jobs.append(job)
            self.env.process(job.main())

In [30]:
def run(seed: int | None) -> System:
    random.seed(seed)
    production_system = System(
        env=simpy.Environment(),
        inter_arrival_time_distribution=lambda: random.expovariate(lambd=0.65),
        processing_time_per_family_distribution=[
            lambda: random.gammavariate(2,2),
            lambda: random.gammavariate(4,0.5),
            lambda: random.gammavariate(6,1/6)
        ],
        families_distribution=lambda: random.random(),
        routing_distribution={
            1: [lambda: random.random(), lambda: random.random(), lambda: random.random(),lambda: random.random(),lambda: random.random(),lambda: random.random()],
            2: [lambda: random.random(), lambda: random.random(), lambda: random.random(),lambda: random.random(),lambda: random.random(),lambda: random.random()],
            3: [lambda: random.random(), lambda: random.random(), lambda: random.random(),lambda: random.random(),lambda: random.random(),lambda: random.random()]
        },
        routing_prob={
            1: [1,1,0,1,1,1],
            2: [0.8, 0.8, 1, 0.8, 0.8, 0.75],
            3: [0,0,1,0,0,0.75]
        },
        due_dates_distribution=lambda: random.uniform(30,50)
    )
    production_system.env.run(until=60*120)
    return production_system

In [31]:
manufacturing_system = run(seed=42)

PRODUCT 0 of family F1: Waiting for the machine WC1 to become available
PRODUCT 0 of family F1: Machine WC1 ready
PRODUCT 0 of family F1: starting to be processed by the machine WC1
PRODUCT 1 of family F2: Waiting for the machine WC1 to become available
PRODUCT 0 of family F1: Machine WC1 process finished
PRODUCT 0 of family F1: Waiting for the machine WC2 to become available
PRODUCT 0 of family F1: Machine WC2 ready
PRODUCT 0 of family F1: starting to be processed by the machine WC2
PRODUCT 1 of family F2: Machine WC1 ready
PRODUCT 1 of family F2: starting to be processed by the machine WC1
PRODUCT 1 of family F2: Machine WC1 process finished
PRODUCT 1 of family F2: Waiting for the machine WC2 to become available
PRODUCT 2 of family F3: Waiting for the machine WC3 to become available
PRODUCT 2 of family F3: Machine WC3 ready
PRODUCT 2 of family F3: starting to be processed by the machine WC3
PRODUCT 2 of family F3: Machine WC3 process finished
PRODUCT 2 of family F3: Waiting for the m

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

